In [1]:
import os, sys

In [2]:
base_dir = os.path.split(os.getcwd())[0]
os.chdir(os.path.join(base_dir,'core'))

In [3]:
import cv2
import numpy as np
import pickle as pk
from copy import deepcopy
import matplotlib.pyplot as plt

import scipy
from scipy.spatial.transform import Rotation as R

from utils import ARUCO_DICT
# from pose_estimation import pose_esitmation
# from utils import ARUCO_DICT

Этот блокнот генерирует маркеры и матрицы их систем координат в соответствии с их расположением
(центра маркеров расположены в 0 по Y и 0 по Z базовой с.к. координата X центров маркеров кратна 8-ми,
ось Х с.к. маркеров совпадает с осью Х с.к. базы, с.к. маркеров повернута относительно с.к. базы вокруг оси Х на -90 градусов)

С.к. базы: ось Х направлена вправо, ось Y против направления взгляда, ось Z вниз.

In [14]:
savedir = '../tags'
dict_type = 'DICT_5X5_50'
marker_size = 10000
n_markers = 31
step = 10

In [16]:
# Генерация и сохранение маркеров

if ARUCO_DICT.get(dict_type, None) is None:
	print(f"ArUCo tag type '{dict_type}' is not supported")
	sys.exit(0)

arucoDict = cv2.aruco.Dictionary_get(ARUCO_DICT[dict_type])

for id in range(n_markers):
    print("Generating ArUCo tag of type '{}' with ID '{}'".format(dict_type, id))
    tag = np.zeros((marker_size, marker_size, 1), dtype="uint8")
    cv2.aruco.drawMarker(arucoDict, id, marker_size, tag, 1)
    tag = cv2.bitwise_not(tag)
    # Save the tag generated
    tag_name = f'{savedir}/{dict_type}_id_{id}.png'
    cv2.imwrite(tag_name, tag)

Generating ArUCo tag of type 'DICT_5X5_50' with ID '0'
Generating ArUCo tag of type 'DICT_5X5_50' with ID '1'
Generating ArUCo tag of type 'DICT_5X5_50' with ID '2'
Generating ArUCo tag of type 'DICT_5X5_50' with ID '3'
Generating ArUCo tag of type 'DICT_5X5_50' with ID '4'
Generating ArUCo tag of type 'DICT_5X5_50' with ID '5'
Generating ArUCo tag of type 'DICT_5X5_50' with ID '6'
Generating ArUCo tag of type 'DICT_5X5_50' with ID '7'
Generating ArUCo tag of type 'DICT_5X5_50' with ID '8'
Generating ArUCo tag of type 'DICT_5X5_50' with ID '9'
Generating ArUCo tag of type 'DICT_5X5_50' with ID '10'
Generating ArUCo tag of type 'DICT_5X5_50' with ID '11'
Generating ArUCo tag of type 'DICT_5X5_50' with ID '12'
Generating ArUCo tag of type 'DICT_5X5_50' with ID '13'
Generating ArUCo tag of type 'DICT_5X5_50' with ID '14'
Generating ArUCo tag of type 'DICT_5X5_50' with ID '15'
Generating ArUCo tag of type 'DICT_5X5_50' with ID '16'
Generating ArUCo tag of type 'DICT_5X5_50' with ID '17'
Ge

In [65]:
# Вычисление систем координат маркеров в системе коррдинат базы

X, Y, Z = (np.linspace(0, n_markers*step, n_markers, endpoint=False), np.zeros(n_markers), np.zeros(n_markers))

def M_X(a):
    c, s = np.cos(a), np.sin(a)
    return np.array([[1,  0,  0],
                     [0,  c, -s],
                     [0,  s,  c]])

def M_Y(a):
    c, s = np.cos(a), np.sin(a)
    return np.array([[c, -s,  0],
                     [s,  c,  0],
                     [0,  0,  1]])

def M_Z(a):
    c, s = np.cos(a), np.sin(a)
    return np.array([[ c,  0,  s],
                     [ 0,  1,  0],
                     [-s,  0,  c]])

a = -90 * 3.1415/180 # угол поворота 
M = M_X(a) # только одно вращение вокруг оси X

In [66]:
marker_poses = [np.vstack([np.column_stack([M, np.array([x, y, z])]), [0,0,0,1]]) for x, y, z in zip(X, Y, Z)]

In [67]:
marker_poses_dict = dict(zip(range(n_markers), marker_poses))

In [68]:
marker_poses_dict

{0: array([[ 1.00000000e+00,  0.00000000e+00,  0.00000000e+00,
          0.00000000e+00],
        [ 0.00000000e+00,  4.63267949e-05,  9.99999999e-01,
          0.00000000e+00],
        [ 0.00000000e+00, -9.99999999e-01,  4.63267949e-05,
          0.00000000e+00],
        [ 0.00000000e+00,  0.00000000e+00,  0.00000000e+00,
          1.00000000e+00]]),
 1: array([[ 1.00000000e+00,  0.00000000e+00,  0.00000000e+00,
          2.09500000e+00],
        [ 0.00000000e+00,  4.63267949e-05,  9.99999999e-01,
          0.00000000e+00],
        [ 0.00000000e+00, -9.99999999e-01,  4.63267949e-05,
          0.00000000e+00],
        [ 0.00000000e+00,  0.00000000e+00,  0.00000000e+00,
          1.00000000e+00]]),
 2: array([[ 1.00000000e+00,  0.00000000e+00,  0.00000000e+00,
          4.19000000e+00],
        [ 0.00000000e+00,  4.63267949e-05,  9.99999999e-01,
          0.00000000e+00],
        [ 0.00000000e+00, -9.99999999e-01,  4.63267949e-05,
          0.00000000e+00],
        [ 0.00000000e+00,  0.0

In [69]:
with open(f"{savedir}/marker_poses.pk", "wb") as f:
    pk.dump(marker_poses_dict, f)